In [ ]:
model_path='./models/facemask.pth' ##Please enter path to the trained model
device='cpu'  ##Please enter device

In [ ]:
#imports
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision

In [ ]:
#CNNs
class Model(nn.Module):
    def __init__(self, height=218, width=178, dim_output=2):
        super(Model, self).__init__()
        
        self.H=height
        self.W=width
        self.output=dim_output
        
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.activation = nn.ReLU()
        self.dropout= nn.Dropout(p=0.5) 
        self.bn1= nn.BatchNorm2d(32) 
        self.bn2= nn.BatchNorm2d(64)
        self.bn3= nn.BatchNorm2d(128)
        
        #convolutions
        self.conv1 = nn.Conv2d( 3,  32,kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32,  64,kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128,kernel_size=3, stride=1, padding=1)
        
        #fully connected
        self.fc1 = nn.Linear(128*(self.H//8)*(self.W//8),128) 
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,self.output)
        
        #sequential
        self.layer1=nn.Sequential(self.conv1, self.bn1, self.max_pool, self.activation)
        self.layer2=nn.Sequential(self.conv2, self.bn2, self.max_pool, self.activation)
        self.layer3=nn.Sequential(self.conv3, self.bn3, self.max_pool, self.activation)
        self.layer4=nn.Sequential(self.fc1, self.activation, self.dropout)
        self.layer5=nn.Sequential(self.fc2, self.activation, self.dropout)
        
    def forward(self, img):
        
        out=self.layer3(self.layer2(self.layer1(img)))
        
        ###flattening
        batch_size =img.shape[0]
        out=out.view(batch_size, -1)
        
        out=self.fc3(self.layer5(self.layer4(out)))
       
        return out
    
model=Model()    

In [ ]:
#Load model
model_load=torch.load(model_path ,map_location=torch.device(device))
model.load_state_dict(model_load['final_model_state_dict'])

In [ ]:
####################Please enter path to face and eye xml files#############################
face_path=os.path.dirname(cv2.__file__)+'/data/haarcascade_frontalface_default.xml'
eye_path =os.path.dirname(cv2.__file__)+'/data/haarcascade_eye.xml'

face_cascade = cv2.CascadeClassifier(face_path)
eye_cascade = cv2.CascadeClassifier(eye_path)

threshold = 80
font = cv2.FONT_HERSHEY_SIMPLEX
org = (30, 30)
mask_font_color = (0, 255, 0)  #green
nomask_font_color = (0, 0, 255) #red
noface_font_color=(250, 206, 135) #light blue
thickness = 2
font_scale = 1
mask = "Thank you for wearing a mask"
#nomask = "Please wear a mask"
cap = cv2.VideoCapture(0)

while True:
    (a , img) = cap.read()
    img = cv2.flip(img,1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (thresh, bw) = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    faces_gray = face_cascade.detectMultiScale(gray, 1.1, 4)
    faces_bw = face_cascade.detectMultiScale(bw, 1.1, 4)
    eyes = eye_cascade.detectMultiScale(gray)
    
    if(len(faces_gray) == 0 and len(faces_bw) == 0 and len(eyes) == 0):
        cv2.putText(img, "No face found", org, font, font_scale, noface_font_color, thickness, cv2.LINE_AA)
    else:
        if(len(faces_gray) == 0 and len(faces_bw) == 0 and len(eyes) != 0):
            cv2.putText(img, mask, org, font, font_scale, mask_font_color, thickness, cv2.LINE_AA)
        else:
            for (x, y, w, h) in faces_gray:
                
               
                face_img = img[y:y+h, x:x+w]
                rerect_sized=cv2.resize(face_img,(218,178))
                normalized=rerect_sized/255.0
                reshaped=np.reshape(normalized,(3,218,178))
                reshaped = np.vstack([reshaped])
                to_tensor=torchvision.transforms.ToTensor()
                t_reshaped=to_tensor(reshaped)
                tensor_reshaped=t_reshaped.unsqueeze(1).permute(1,2,3,0)
                result=model(tensor_reshaped.float())
                label1=torch.argmax(result, dim=1)[0]
                label=label1.detach().item()
                
                if label == 1:
                    cv2.putText(img, mask, org, font, font_scale, mask_font_color, thickness, cv2.LINE_AA)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                else:
                    cv2.putText(img, "no mask", (x, y-10), font, font_scale, nomask_font_color, thickness, cv2.LINE_AA)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    cv2.imshow('LIVE',   img)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()          